# Cleaning and preliminary exploration of customers dataset

## imports 

In [1]:
import os, sys, glob # for loading, saving and manipulating files on disk
from pathlib import Path # Path allows us to list directory paths in a manner that's not affected by the OS the notebook is being run on 

import pandas as pd # to work with csv files as dataframes 
import numpy as np # for mathematical and scientific computations 

import matplotlib.pyplot as plt # for plots 
import seaborn as sns # also for plotting, it's a wrapper for several matplotlib functions, making them more accessible 

from geopy import GoogleV3
api_key = os.environ['gmap_api_key']

## loading data

In [2]:
data_dir = Path.home()/'projects'/'billups_data_challenge'/'Data Challenge' # specifies the directory with data files
out_dir = data_dir.parent/'cleaned_data' # directory for storing intermediate outputs such as cleaned data
out_dir.mkdir(exist_ok=True) #creating the output directory, while making sure it's not overwritten if it already exists

In [3]:
customers_df = pd.read_csv(data_dir/'customers.csv')

## inspecting the dataset

In [4]:
customers_df.shape

(2000, 10)

In [5]:
customers_df.sample(10)

id              name  \
1991  773e32e4eac2ec735b84cd568ec4fd05        Kirk White   
430   397b94d4ed0b57316442dc469535bcc1      Dennis Hayes   
358   1302a5932197b899a6e047296ebadc6c       Amelia Bush   
1749  e3e301280fdf5073bbaefb03f3a49eb7         John Hood   
379   9b6f084dd52711f78b8ad036a79d355c        Elisa Venn   
259   5d0b73c12d17deb7ff692dd8ff4dfb38      Robert Hobbs   
854   79aba093cc41da3ae041e263e19026e4    Delila Ritchie   
1638  448ab9c5979a7d104f51d334d59159e3  Linda Saunderson   
104   ad586224603a96533e48b2e74635ea51     PAMELA THOMAS   
1278  d98ec870a1873422af418ad59347844b     Marion Krause   

            street_address      city   state   zipcode  birth_date  gender  \
1991      4412 NE 18th Ave  Portland  Oregon   97211.0  1956-07-19    male   
430        6028 SW Raab Rd  Portland  Oregon   97221.0  1952-05-14    male   
358    1808 SW Morrison St  Portland  Oregon   97205.0  2006-04-13  female   
1749    3943 SE Madison St  Portland  Oregon   97214.0  1960-08-05    male   
379       7007 SE 36th Ave  Portland  Oregon   97202.0  2001-03-07     NaN   
259        566 NE Royal ct       NaN  Oregon  -97232.0  2004-01-02    male   
854       5604 SE 57th Ave  Portland     NaN   97206.0  1976-03-10  female   
1638     4620 SW Wood Pkwy  Portland  Oregon   97219.0  2006-09-15  female   
104     2611 NE Alameda St  Portland  Oregon   97212.0  1988-02-18     NaN   
1278  6857 NE Alderwood Rd  Portland  Oregon   97218.0  1998-04-18  female   

            education          occupation  
1991           Others              Others  
430            Others              Others  
358           College               Sales  
1749              NaN                Tech  
379       High School  Education & Health  
259            Others     Admin & Support  
854   Graduate Degree  Education & Health  
1638          College               Sales  
104       High school     Admin & Support  
1278     High  School              Others

In [6]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2000 non-null   object
 1   name            2000 non-null   object
 2   street_address  2000 non-null   object
 3   city            1906 non-null   object
 4   state           1898 non-null   object
 5   zipcode         1999 non-null   object
 6   birth_date      1893 non-null   object
 7   gender          1899 non-null   object
 8   education       1896 non-null   object
 9   occupation      1893 non-null   object
dtypes: object(10)
memory usage: 156.4+ KB


## cleaning addresses and geocoding them 

### filling in missing values for city and state

In [7]:
customers_df['city'].fillna('Portland', inplace=True)
customers_df['state'].fillna('Oregon', inplace=True)

### cleaning up zipcodes

In [8]:
# removing non-numeric characters
customers_df['zipcode'] = customers_df.zipcode.str.replace(r'[^\d\.]', '', regex=True).str.strip().str.replace('.0', '', regex=False)

### replacing "NaN"s with empty strings

In [9]:
customers_df[['street_address', 'city', 'state', 'zipcode']] = customers_df[['street_address', 'city', 'state', 'zipcode']].apply(
lambda x: x.str.replace('nan', ''))

### generating full address

In [10]:
customers_df['full_address'] = customers_df['street_address']+' '+customers_df['city']+' '+customers_df['state']+ ' '+customers_df['zipcode']
customers_df['full_address'].fillna('', inplace=True)

## geocoding addresses

In [11]:
import requests

In [12]:
addresses = customers_df['full_address'].tolist()

In [13]:
def geocodeAPI_results(address, key, return_full_response=True):
    geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(address)
    if key is not None: 
        geocode_url = geocode_url + '&key={}'.format(key)
    geocoding_results = requests.get(geocode_url)
    geocoding_results = geocoding_results.json()
    if len(geocoding_results['results']) == 0: 
        output ={
            'geocode_formatted_address': None, 
            'geocode_lat': None, 
            'geocode_lng': None
        }
    else: 
        answer = geocoding_results['results'][0]
        output = {
            'geocode_formatted_address': answer.get('formatted_address'),
            'geocode_lat': answer.get('geometry').get('location').get('lat'), 
            'geocode_lng': answer.get('geometry').get('location').get('lng'),
        }
        
    output['geocode_input_string'] = address
    output['geocode_number_of_results'] = len(geocoding_results['results'])
    output['geocode_status'] = geocoding_results.get('status')
    if return_full_response is True: 
        output['geocode_response'] = geocoding_results
    return output

In [14]:
results = []
for address in addresses: 
    geocoded = False
    while geocoded is not True: 
        try: 
            geocoded_result = geocodeAPI_results(address, api_key, return_full_response=False)
        except Exception as e:
            print(e)
        geocoded=True
        results.append(geocoded_result)
        

In [15]:
results_df = pd.DataFrame(results)
results_df.rename(columns={'geocode_formatted_address':'geocoded_address', 'geocode_lat':'lat',
                           'geocode_lng': 'long', 'geocode_input_string':'full_address'}, inplace=True)
customers_df = customers_df.merge(results_df[['full_address', 'geocoded_address', 'lat', 'long']], on='full_address', how='left')

In [16]:
customers_df

id               name  \
0     df83ec2d0d409395c0d8c2690cfa8b67  Cynthia Barnfield   
1     6aec7ab2ea0d67161dac39e5dcabd857    Elizabeth Smith   
2     0c54340672f510fdb9d2f30595c1ab53      Richard Pabla   
3     f0d9ce833ddc1f73c1e0b55bdebf012e      Charles Baker   
4     3720379163f6b46944db6c98c0485bfd       Ronald Lydon   
...                                ...                ...   
2471  9a2194fcd4f0f326f0ca334450e16a93         Earl Grier   
2472  01a598a05c48fdd18461d6411f51a109   Rogelio Richmann   
2473  01a598a05c48fdd18461d6411f51a109   Rogelio Richmann   
2474  01a598a05c48fdd18461d6411f51a109   Rogelio Richmann   
2475  2d1d65505400e568a045c828c9ebe365         Paul Groat   

            street_address       city   state zipcode  birth_date   gender  \
0          44 NE Meikle Pl  Portland   Oregon   97213  2009-09-10   female   
1      7511 SE Harrison St   Portland  Oregon   97215  1956-12-15   female   
2          1404 SE Pine St   Portland  Oregon   97214  1960-12-18     male   
3     12271 N Westshore Dr   Portland  Oregon   97217  2105-07-19     male   
4      5321 NE Skyport Way   Portland  Oregon   97218  1961-03-14     male   
...                    ...        ...     ...     ...         ...      ...   
2471       22 NE graham ST   Portland  OREGON   97212  2007-10-02     male   
2472   7000 NE Airport Way   Portland  OREGON   97218  2001-02-19     male   
2473   7000 NE Airport Way   Portland  OREGON   97218  2001-02-19     male   
2474   7000 NE Airport Way   Portland  OREGON   97218  2001-02-19     male   
2475       8993 SE 78th Pl   Portland  Oregon   97206         NaN     male   

            education          occupation  \
0         High School                 NaN   
1             College         Blue Collar   
2             College  Education & Health   
3     Graduate Degree               SALES   
4     Graduate Degree         Blue Collar   
...               ...                 ...   
2471           Others  Education & Health   
2472          College  Business & Finance   
2473          College  Business & Finance   
2474          College  Business & Finance   
2475           Others               Sales   

                                    full_address  \
0         44 NE Meikle Pl Portland  Oregon 97213   
1     7511 SE Harrison St  Portland Oregon 97215   
2          1404 SE Pine St Portland Oregon 97214   
3     12271 N Westshore Dr Portland Oregon 97217   
4      5321 NE Skyport Way Portland Oregon 97218   
...                                          ...   
2471       22 NE graham ST Portland OREGON 97212   
2472   7000 NE Airport Way Portland OREGON 97218   
2473   7000 NE Airport Way Portland OREGON 97218   
2474   7000 NE Airport Way Portland OREGON 97218   
2475       8993 SE 78th Pl Portland Oregon 97206   

                                   geocoded_address        lat        long  
0          44 NE Meikle Pl, Portland, OR 97213, USA  45.523487 -122.618735  
1      7511 SE Harrison St, Portland, OR 97215, USA  45.509127 -122.586057  
2          1404 SE Pine St, Portland, OR 97214, USA  45.520622 -122.651425  
3     12271 N Westshore Dr, Portland, OR 97217, USA  45.613485 -122.692632  
4      5321 NE Skyport Way, Portland, OR 97218, USA  45.576465 -122.609121  
...                                             ...        ...         ...  
2471       22 NE Graham St, Portland, OR 97212, USA  45.542684 -122.665952  
2472   7000 NE Airport Way, Portland, OR 97218, USA  45.589159 -122.593494  
2473   7000 NE Airport Way, Portland, OR 97218, USA  45.589159 -122.593494  
2474   7000 NE Airport Way, Portland, OR 97218, USA  45.589159 -122.593494  
2475       8993 SE 78th Pl, Portland, OR 97206, USA  45.457031 -122.583487  

[2476 rows x 14 columns]

## missing values 

In [17]:
customers_df.isna().sum()

id                    0
name                  0
street_address        0
city                  0
state                 0
zipcode               1
birth_date          128
gender              123
education           136
occupation          135
full_address          0
geocoded_address      1
lat                   1
long                  1
dtype: int64

## cleaning birth dates and generating age variable 

In [18]:
customers_df.birth_date.fillna('', inplace=True)

In [19]:
customers_df['birth_date'] = customers_df['birth_date'].apply(lambda x: pd.to_datetime(x.replace(r'\D+', ''), format='%Y-%m-%d'))

In [20]:
customers_df['birth_year'] = customers_df['birth_date'].dt.year


- we have 99 observations where the birth year is in the next century
- while at first glance, it may seem like there is a simple typo which can be corrected with changing the 100s digit from 1 to 0, that is incorrect, as there exist patrons born in '2118' (for example) that have an education level of 'High School' which is inconsistent with 2018. 
- it's unclear here, what the correct imputation would be, therefore I drop these values 

In [22]:
customers_df['birth_year'].mask(customers_df['birth_year']>2022, np.nan, inplace=True)

## saving to disk 

In [23]:
customers_df.to_csv(out_dir/'customers_cleaned_geocoded.csv', index=False)